In [2]:
library(Signac)
library(Seurat)
library(GenomeInfoDb)
library(ggplot2)
library(patchwork)
library(stringr)
set.seed(1234)

Attaching SeuratObject

Attaching sp

Lade nötiges Paket: BiocGenerics


Attache Paket: ‘BiocGenerics’


Die folgenden Objekte sind maskiert von ‘package:stats’:

    IQR, mad, sd, var, xtabs


Die folgenden Objekte sind maskiert von ‘package:base’:

    Filter, Find, Map, Position, Reduce, anyDuplicated, append,
    as.data.frame, basename, cbind, colnames, dirname, do.call,
    duplicated, eval, evalq, get, grep, grepl, intersect, is.unsorted,
    lapply, mapply, match, mget, order, paste, pmax, pmax.int, pmin,
    pmin.int, rank, rbind, rownames, sapply, setdiff, sort, table,
    tapply, union, unique, unsplit, which.max, which.min


Lade nötiges Paket: S4Vectors

Lade nötiges Paket: stats4


Attache Paket: ‘S4Vectors’


Die folgenden Objekte sind maskiert von ‘package:base’:

    I, expand.grid, unname


Lade nötiges Paket: IRanges


Attache Paket: ‘IRanges’


Das folgende Objekt ist maskiert ‘package:sp’:

    %over%




In [3]:
seurat <- readRDS("/Users/dominik.klein/data/E14-E15_adata_filtered_rmDoublets_seurat.rds")

In [4]:
new_path_1 = "/Users/dominik.klein/data/pancreas_multiome/E14-5/outs/atac_fragments.tsv.gz"

In [5]:
new_path_2 = "/Users/dominik.klein/data/pancreas_multiome/E15_5/outs/atac_fragments.tsv.gz"

In [6]:
res <- UpdatePath(Fragments(seurat)[[1]], new_path_1 , verbose = TRUE)

Computing hash



In [7]:
res2 <- UpdatePath(Fragments(seurat)[[2]], new_path_2 , verbose = TRUE)

Computing hash



In [8]:
tmp <- Fragments(seurat)

In [9]:
tmp[[1]] <- NULL

In [10]:
tmp[[1]] <- res

In [11]:
tmp[[2]] <- NULL

In [12]:
tmp[[2]] <- res2

In [13]:
seurat <- SetAssayData(seurat, slot = "fragments", new.data = tmp)

Warning message in SetAssayData.ChromatinAssay(object = object[[assay]], slot = slot, :
“Overwriting existing fragment objects”


In [14]:
merged_peaks <- granges(seurat[["ATAC"]])
merged_peaks <- keepStandardChromosomes(merged_peaks, pruning.mode = "coarse")

In [15]:
cr_peaks <- FeatureMatrix(
  fragments = Fragments(seurat),
  features = merged_peaks,
  cells = colnames(seurat)
)

Extracting reads overlapping genomic regions

Extracting reads overlapping genomic regions



In [18]:
dim(cr_peaks)

[1] 228259  16978

In [19]:
seurat[["ATAC_merged"]] <- CreateChromatinAssay(
  counts = cr_peaks,
  fragments = Fragments(seurat),
  annotation = seurat[["ATAC"]]@annotation
)

Warning message:
“Keys should be one or more alphanumeric characters followed by an underscore, setting key from atac_merged_ to atacmerged_”


In [20]:
seurat

An object of class Seurat 
487835 features across 16978 samples within 3 assays 
Active assay: ATAC (228321 features, 0 variable features)
 2 other assays present: RNA, ATAC_merged

In [85]:
saveRDS(seurat, "/Users/dominik.klein/data/E14-E15_adata_filtered_seurat.rds")